The following python code will create a STAC catalog from a local geotif raster


In [20]:
%pip install boto3 rasterio shapely pystac

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pystac
import os

In [22]:
catalog = pystac.Catalog(id='test-catalog', description='Tutorial catalog.')

In [23]:
import rasterio
from shapely.geometry import Polygon, mapping

def get_bbox_and_footprint(raster_uri):
    with rasterio.open(raster_uri) as ds:
        bounds = ds.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        
        return (bbox, mapping(footprint))

In [24]:
img_path="/home/jgillan/Documents/STAC_drone/1a_g2_ortho.tif"

bbox, footprint = get_bbox_and_footprint(img_path)
print(bbox)
print(footprint)



[-110.88731036534, 31.8109444965928, -110.884672430915, 31.8130591450936]
{'type': 'Polygon', 'coordinates': (((-110.88731036534, 31.8109444965928), (-110.88731036534, 31.8130591450936), (-110.884672430915, 31.8130591450936), (-110.884672430915, 31.8109444965928), (-110.88731036534, 31.8109444965928)),)}


In [25]:
from datetime import datetime

item = pystac.Item(id='local-image',
                 geometry=footprint,
                 bbox=bbox,
                 datetime=datetime.utcnow(),
                 properties={})

In [26]:
assert item.get_parent() is None

In [27]:
catalog.add_item(item)

In [28]:
item.get_parent()

ID: test-catalog
Description: Tutorial catalog.
ID: local-image
"Bounding Box: [-110.88731036534, 31.8109444965928, -110.884672430915, 31.8130591450936]"
Datetime: 2022-12-15 20:15:27.441453
Rel: root
Target:
Media Type: application/json
Rel: parent
Target:
Media Type: application/json


In [29]:
catalog.describe()

* <Catalog id=test-catalog>
  * <Item id=local-image>


In [30]:
item.add_asset(
    key='image', 
    asset=pystac.Asset(
        href=img_path, 
        media_type=pystac.MediaType.GEOTIFF
    )
)

In [31]:
import json
print(json.dumps(item.to_dict(), indent=4))

{
    "type": "Feature",
    "stac_version": "1.0.0",
    "id": "local-image",
    "properties": {
        "datetime": "2022-12-15T20:15:27.441453Z"
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    -110.88731036534,
                    31.8109444965928
                ],
                [
                    -110.88731036534,
                    31.8130591450936
                ],
                [
                    -110.884672430915,
                    31.8130591450936
                ],
                [
                    -110.884672430915,
                    31.8109444965928
                ],
                [
                    -110.88731036534,
                    31.8109444965928
                ]
            ]
        ]
    },
    "links": [
        {
            "rel": "root",
            "href": null,
            "type": "application/json"
        },
        {
            "rel": "parent",
 

In [32]:
print(catalog.get_self_href() is None)
print(item.get_self_href() is None)

True
True


In [33]:
stac_directory="/home/jgillan/Documents/STAC_drone"

catalog.normalize_hrefs(os.path.join(stac_directory, 'stac'))

In [34]:
print(catalog.get_self_href())
print(item.get_self_href())

/home/jgillan/Documents/STAC_drone/stac/catalog.json
/home/jgillan/Documents/STAC_drone/stac/local-image/local-image.json


In [35]:
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [36]:
with open(item.self_href) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2022-12-15T20:15:27.441453Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -110.88731036534,
          31.8109444965928
        ],
        [
          -110.88731036534,
          31.8130591450936
        ],
        [
          -110.884672430915,
          31.8130591450936
        ],
        [
          -110.884672430915,
          31.8109444965928
        ],
        [
          -110.88731036534,
          31.8109444965928
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json"
    }
  ],
  "assets": {
    "image": {
      "href": "/home/jgillan/Documents/STAC_drone/1a_g2_ortho.tif",
      "type": "image/tiff; application=geotiff"
    }
  },
  "bbox": [
   

In [37]:
with open(item.self_href) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2022-12-15T20:15:27.441453Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -110.88731036534,
          31.8109444965928
        ],
        [
          -110.88731036534,
          31.8130591450936
        ],
        [
          -110.884672430915,
          31.8130591450936
        ],
        [
          -110.884672430915,
          31.8109444965928
        ],
        [
          -110.88731036534,
          31.8109444965928
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json"
    }
  ],
  "assets": {
    "image": {
      "href": "/home/jgillan/Documents/STAC_drone/1a_g2_ortho.tif",
      "type": "image/tiff; application=geotiff"
    }
  },
  "bbox": [
   

In [38]:
catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)